# California Rebates — 2024 Results (Household-level)

Scope: California households, calendar year 2024. We extend the calibrated household panel (Step 01) to evaluate two alternative rebate formulas. All results are **household-weighted**, restricted to AGI ≥ 0 households, and grouped by equivalized income (AGI ÷ household size).

---

## Policy Scenarios

### 1. Poverty × VAT Rebate
- **Base**: `poverty_threshold(hh_size) × vat_rate`  
- **Phase-out**: full base for AGI ≤ 150% of poverty, linearly to zero by 200%  
- **No marriage adjustment** (only household size matters)

**Key 2024 results**
- No phase-out total: (see `summary_2024.csv`)  
- With phase-out total: (see `summary_2024.csv`)  
- Reduction from phase-out: difference above  
- Distribution: by decile (`by_decile_2024.csv`) and by household size (`by_size_2024.csv`)  
- Plot: `plots/deciles_2024.png`

---

### 2. Flat Per-Capita Rebate
- **Base**: `$1,700 × household_size`  
- **Phase-out**: starts at $50,000 (non-MFJ) or $100,000 (MFJ); taper = 5¢ per $1  
- **Floored at zero**

**Key 2024 results**
- No phase-out total: (see `summary_2024.csv`)  
- With phase-out total: (see `summary_2024.csv`)  
- Reduction from phase-out: difference above  
- Distribution: by decile (`by_decile_2024.csv`), by household size (`by_size_2024.csv`), and by filing status (`by_status_2024.csv`)  
- Plot: `plots/deciles_2024.png`

---

## Outputs

- Poverty×VAT: `outputs/rebates/poverty_times_vat/`  
  - `rebate_records_2024.csv` — household-level rebates  
  - `summary_2024.csv` — totals with & without phase-out  
  - `by_decile_2024.csv` — totals by equivalized-income decile  
  - `by_size_2024.csv` — totals by household size bucket  
  - `plots/deciles_2024.png` — decile distribution plot

- Flat per-capita: `outputs/rebates/flat_per_capita/`  
  - `rebate_records_2024.csv` — household-level rebates  
  - `summary_2024.csv` — totals with & without phase-out  
  - `by_decile_2024.csv` — totals by equivalized-income decile  
  - `by_size_2024.csv` — totals by household size bucket  
  - `by_status_2024.csv` — totals by filing status  
  - `plots/deciles_2024.png` — decile distribution plot

---

## Method Notes

- Entity = household throughout; we exclude AGI < 0 households  
- Household weights come from Step 01 (already deflated to align with controls)  
- Both formulas enforce:  
  - **Non-negativity** (rebates never below zero)  
  - **Monotone taper** (rebates weakly decline with AGI in phase-out)  
  - **With-phase-out ≤ base** totals for all subgroups and overall  
- Deciles are based on **AGI per capita** (AGI ÷ household size), consistent with prior steps  
- Household size capped at 7+ for the poverty threshold schedule

---

✅ These outputs complete the evaluation of the two proposed rebate structures for 2024.


In [1]:
# 09 — Results rollup (2024 only): Poverty×VAT (07) and Flat per-capita (08)

import os, sys, json, textwrap, subprocess
from pathlib import Path
import pandas as pd
import numpy as np

# --------- SETTINGS ---------
DO_GIT_ACTIONS = False  # set True to auto-create branch + commit
BRANCH_NAME = "feature/rebates-ptv-flat-2024"
# ----------------------------

# ---  repo root 
REPO_ROOT = Path(r"C:\Users\Ali.Melad\Dropbox\Ali Work\Kyle\California VAT\policy_engile_cali_v2")
OUT = REPO_ROOT / "outputs" / "rebates"
DOCS = REPO_ROOT / "docs"
DOCS.mkdir(parents=True, exist_ok=True)

PTV = OUT / "poverty_times_vat"
FLAT = OUT / "flat_per_capita"

expected_files = {
    # Poverty × VAT (07)
    "ptv_records":  PTV / "rebate_records_2024.csv",
    "ptv_summary":  PTV / "summary_2024.csv",
    "ptv_decile":   PTV / "by_decile_2024.csv",
    "ptv_size":     PTV / "by_size_2024.csv",
    # Flat per-capita (08)
    "flat_records": FLAT / "rebate_records_2024.csv",
    "flat_summary": FLAT / "summary_2024.csv",
    "flat_decile":  FLAT / "by_decile_2024.csv",
    "flat_size":    FLAT / "by_size_2024.csv",
    "flat_status":  FLAT / "by_status_2024.csv",
}

# 1) Check files exist
missing = [k for k, p in expected_files.items() if not p.exists()]
if missing:
    print("⚠️ Missing outputs:", missing)
    print("Run 07_poverty_times_vat_rebate and 08_flat_per_capita_rebate first.")
else:
    print("✅ All expected output files found for 07 & 08.")

# 2) Load and summarize
def read_csv(path: Path):
    try:
        return pd.read_csv(path)
    except Exception as e:
        raise RuntimeError(f"Failed reading {path}: {e}")

tables = {k: read_csv(p) for k, p in expected_files.items() if p.exists()}

def money(x):
    try:
        return f"${float(x):,.0f}"
    except:
        return str(x)

def pct(x):
    try:
        return f"{100*float(x):.2f}%"
    except:
        return str(x)

# --- Poverty × VAT (07) ---
if "ptv_summary" in tables:
    s = tables["ptv_summary"].iloc[0]
    ptv_rate = s.get("vat_rate", float("nan"))
    ptv_with = s.get("total_with_phaseout", float("nan"))
    ptv_base = s.get("total_no_phaseout", float("nan"))
    print("\n--- Poverty × VAT (2024) ---")
    print("VAT rate:", ptv_rate)
    print("No phase-out total:", money(ptv_base))
    print("With phase-out total:", money(ptv_with))
    print("Reduction from phase-out:", money(ptv_base - ptv_with))

if "ptv_decile" in tables:
    dec = tables["ptv_decile"].copy()
    dec["weighted_total_fmt"] = dec["weighted_total"].map(money)
    print("\nPoverty×VAT — totals by equivalized-income decile:")
    print(dec[["decile", "weighted_total_fmt"]].to_string(index=False))

if "ptv_size" in tables:
    sz = tables["ptv_size"].copy()
    sz["weighted_total_fmt"] = sz["weighted_total"].map(money)
    print("\nPoverty×VAT — totals by household size bucket:")
    print(sz[["size_bucket", "weighted_total_fmt"]].to_string(index=False))

# --- Flat per-capita (08) ---
if "flat_summary" in tables:
    s = tables["flat_summary"].iloc[0]
    amt = s.get("amount", 1700.0)
    ss = s.get("single_start", 50_000.0)
    ms = s.get("mfj_start", 100_000.0)
    rate = s.get("phaseout_rate", 0.05)
    fl_with = s.get("total_with_phaseout", float("nan"))
    fl_base = s.get("total_no_phaseout", float("nan"))
    print("\n--- Flat per-capita (2024) ---")
    print(f"Amount per person: {money(amt)}  •  Single start: {money(ss)}  •  MFJ start: {money(ms)}  •  Phase-out rate: {rate:.3f}")
    print("No phase-out total:", money(fl_base))
    print("With phase-out total:", money(fl_with))
    print("Reduction from phase-out:", money(fl_base - fl_with))

if "flat_status" in tables:
    st = tables["flat_status"].copy()
    st["weighted_total_fmt"] = st["weighted_total"].map(money)
    print("\nFlat per-capita — totals by filing status:")
    print(st[["status_group", "weighted_total_fmt"]].to_string(index=False))

if "flat_decile" in tables:
    dec = tables["flat_decile"].copy()
    dec["weighted_total_fmt"] = dec["weighted_total"].map(money)
    print("\nFlat per-capita — totals by equivalized-income decile:")
    print(dec[["decile", "weighted_total_fmt"]].to_string(index=False))

if "flat_size" in tables:
    sz = tables["flat_size"].copy()
    sz["weighted_total_fmt"] = sz["weighted_total"].map(money)
    print("\nFlat per-capita — totals by household size bucket:")
    print(sz[["size_bucket", "weighted_total_fmt"]].to_string(index=False))

# 3) Write docs/rebates_summary_2024.md
ptv_vals = tables.get("ptv_summary", pd.DataFrame([{}])).iloc[0].to_dict() if "ptv_summary" in tables else {}
flat_vals = tables.get("flat_summary", pd.DataFrame([{}])).iloc[0].to_dict() if "flat_summary" in tables else {}

ptv_no   = money(ptv_vals.get("total_no_phaseout", "n/a"))
ptv_with = money(ptv_vals.get("total_with_phaseout", "n/a"))
ptv_red  = "n/a"
try:
    ptv_red = money(ptv_vals.get("total_no_phaseout", 0) - ptv_vals.get("total_with_phaseout", 0))
except: pass

flat_no   = money(flat_vals.get("total_no_phaseout", "n/a"))
flat_with = money(flat_vals.get("total_with_phaseout", "n/a"))
flat_red  = "n/a"
try:
    flat_red = money(flat_vals.get("total_no_phaseout", 0) - flat_vals.get("total_with_phaseout", 0))
except: pass

readme = textwrap.dedent(f"""\
# Rebate Results — 2024 (Household-level)

This page summarizes outputs from:

- **07_poverty_times_vat_rebate.ipynb** → `outputs/rebates/poverty_times_vat/`
- **08_flat_per_capita_rebate.ipynb** → `outputs/rebates/flat_per_capita/`

## Poverty × VAT (2024)

- VAT rate: {ptv_vals.get('vat_rate', 'n/a')}
- No phase-out total: {ptv_no}
- With phase-out total: {ptv_with}
- Reduction from phase-out: {ptv_red}

Files:
- Records: `outputs/rebates/poverty_times_vat/rebate_records_2024.csv`
- Summary: `outputs/rebates/poverty_times_vat/summary_2024.csv`
- By decile: `outputs/rebates/poverty_times_vat/by_decile_2024.csv`
- By size: `outputs/rebates/poverty_times_vat/by_size_2024.csv`
- Plot: `outputs/rebates/poverty_times_vat/plots/deciles_2024.png`

## Flat per-capita (2024)

- Amount per person: {money(flat_vals.get('amount', 1700))}
- Single start: {money(flat_vals.get('single_start', 50_000))}
- MFJ start: {money(flat_vals.get('mfj_start', 100_000))}
- Phase-out rate: {flat_vals.get('phaseout_rate', 0.05)}
- No phase-out total: {flat_no}
- With phase-out total: {flat_with}
- Reduction from phase-out: {flat_red}

Files:
- Records: `outputs/rebates/flat_per_capita/rebate_records_2024.csv`
- Summary: `outputs/rebates/flat_per_capita/summary_2024.csv`
- By decile: `outputs/rebates/flat_per_capita/by_decile_2024.csv`
- By size: `outputs/rebates/flat_per_capita/by_size_2024.csv`
- By status: `outputs/rebates/flat_per_capita/by_status_2024.csv`
- Plot: `outputs/rebates/flat_per_capita/plots/deciles_2024.png`

## Notes

- All totals are **household-weighted**. Deciles are computed on **AGI per capita** (AGI ÷ household size).
- Both policies enforce **non-negativity** and **monotone phase-out** by construction.
- Phase-outs ensure **with-phase-out ≤ no-phase-out** totals by group and overall.
""")

out_md = DOCS / "rebates_summary_2024.md"
out_md.write_text(readme, encoding="utf-8")
print("\n✅ Wrote", out_md)

# 4) Optional: create branch and commit
def run(cmd, cwd=None):
    print("$", " ".join(cmd))
    res = subprocess.run(cmd, cwd=cwd, capture_output=True, text=True)
    if res.returncode != 0:
        print(res.stdout)
        print(res.stderr)
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")
    return res.stdout.strip()

def inside_git_repo(cwd: Path):
    try:
        run(["git", "rev-parse", "--is-inside-work-tree"], cwd=cwd)
        return True
    except Exception:
        return False

if DO_GIT_ACTIONS:
    if not inside_git_repo(REPO_ROOT):
        raise RuntimeError("Not inside a git repo; open the project root and rerun with DO_GIT_ACTIONS=True.")

    try:
        run(["git", "checkout", "-b", BRANCH_NAME], cwd=REPO_ROOT)
    except RuntimeError:
        run(["git", "checkout", BRANCH_NAME], cwd=REPO_ROOT)

    to_add = [str(out_md)] + [str(p) for p in expected_files.values() if p.exists()]
    run(["git", "add"] + to_add, cwd=REPO_ROOT)
    run(["git", "commit", "-m", "Rebates (2024): poverty×VAT and flat per-capita results + docs"], cwd=REPO_ROOT)

    print("\n✅ Git commit created on branch:", BRANCH_NAME)
    print("Next steps:")
    print("  git push -u origin", BRANCH_NAME)
    print("  gh pr create --title \"Rebates 2024 (Poverty×VAT & Flat)\" --body \"See docs/rebates_summary_2024.md\"")

# Suggested PR text
pr_title = "Rebates 2024: Poverty×VAT & Flat Per-Capita — results & docs"
pr_body = textwrap.dedent("""\
Adds results and documentation for two rebate designs (household-level, 2024):

- Poverty × VAT: base = poverty threshold × VAT rate; phase-out 150–200% of poverty (AGI basis).
- Flat per-capita: $1,700 × household size; phase-out starts at $50k (non-MFJ) / $100k (MFJ) at 5¢ per $1.

Outputs include record-level rebates, summaries, and decile/size/status breakdowns, with acceptance checks (non-negativity, monotone taper, with-phase-out ≤ base).
""")

print("\n--- Suggested PR ---")
print("Title:", pr_title)
print("Body:\n", pr_body)
print("\n✅ 09 rollup complete.")


✅ All expected output files found for 07 & 08.

--- Poverty × VAT (2024) ---
VAT rate: 0.1
No phase-out total: $33,733,976,971
With phase-out total: $12,508,775,845
Reduction from phase-out: $21,225,201,127

Poverty×VAT — totals by equivalized-income decile:
 decile weighted_total_fmt
      1     $4,341,722,645
      2     $3,837,464,865
      3     $2,913,229,773
      4       $866,405,137
      5       $538,078,290
      6        $11,875,135
      7                 $0
      8                 $0
      9                 $0
     10                 $0

Poverty×VAT — totals by household size bucket:
 size_bucket weighted_total_fmt
           1     $1,888,824,397
           2     $2,935,559,381
           3     $3,676,097,491
           4     $1,812,294,973
           5     $1,556,945,076
           6       $588,309,321
           7        $50,745,206

--- Flat per-capita (2024) ---
Amount per person: $1,700  •  Single start: $50,000  •  MFJ start: $100,000  •  Phase-out rate: 0.050
No pha

In [1]:
# --- VALIDATION CELL: no-phase-out totals & implicit allowances (2024) ---

import sys, json
from pathlib import Path
import numpy as np
import pandas as pd

# ===== Repo + config paths (adjust only if you move the repo) =====
REPO_ROOT = Path(r"C:\Users\Ali.Melad\Dropbox\Ali Work\Kyle\California VAT\policy_engile_cali_v2")
INTERMEDIATE = REPO_ROOT / "intermediate"

# Make repo importable
if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))

# Imports from your policy modules (poverty thresholds + formulas)
from policy.constants import poverty_threshold
from policy.rebates.poverty_times_vat import poverty_times_vat_rebate
from policy.rebates.flat_per_capita import flat_per_capita_rebate

# ----- Load config (VAT rate & flat params) -----
def _load_cfg():
    for p in [REPO_ROOT/"config.json", REPO_ROOT/"config"/"config.json", REPO_ROOT/"config"/"params.json"]:
        if p.exists():
            try:
                return json.loads(Path(p).read_text())
            except Exception:
                pass
    return {}

CFG = _load_cfg()
VAT_RATE = 0.16

FLAT_AMOUNT = CFG.get("rebate", {}).get("flat", {}).get("amount", 1700.0)
PH = CFG.get("rebate", {}).get("flat", {}).get("phaseout", {})
FLAT_SINGLE_START = PH.get("single_start", 50_000.0)
FLAT_MFJ_START = PH.get("mfj_start", 100_000.0)
FLAT_RATE = PH.get("rate", 0.05)

print("Params →", {"vat.rate": VAT_RATE, "flat.amount": FLAT_AMOUNT,
                 "flat.single_start": FLAT_SINGLE_START, "flat.mfj_start": FLAT_MFJ_START, "flat.rate": FLAT_RATE})

# ----- Load Step-01 panel -----
panel = None
for fname in ["ca_panel_2024.parquet","ca_panel_2024.csv",
              "ca_panel_2024_2025.parquet","ca_panel_2024_2025.csv"]:
    p = INTERMEDIATE / fname
    if p.exists():
        panel = pd.read_parquet(p) if p.suffix == ".parquet" else pd.read_csv(p)
        break
if panel is None:
    raise FileNotFoundError(f"Panel not found in {INTERMEDIATE}")

# Coerce types
panel["household_agi"] = pd.to_numeric(panel["household_agi"], errors="coerce").fillna(0.0)
panel["household_size"] = pd.to_numeric(panel["household_size"], errors="coerce").fillna(1).astype(int).clip(lower=1)
panel["household_weight"] = pd.to_numeric(panel["household_weight"], errors="coerce").fillna(0.0)
panel["filing_status"] = panel["filing_status"].astype(str)

def wsum(x, w): return float((x.astype(float) * w.astype(float)).sum())

# ===== A) Poverty × VAT: no-phase-out vs implied thresholds =====
# No-phase-out (rebate dollars): poverty_threshold * VAT_RATE
ptv_base_each = panel["household_size"].map(lambda s: poverty_threshold(int(s)) * float(VAT_RATE))
ptv_base_total = wsum(ptv_base_each, panel["household_weight"])

# "Without VAT multiplication": just the thresholds (consumption allowance dollars)
ptv_threshold_each = panel["household_size"].map(lambda s: poverty_threshold(int(s)))
ptv_threshold_total = wsum(ptv_threshold_each, panel["household_weight"])

# Identity check: ptv_base_total / VAT_RATE ≈ ptv_threshold_total
if VAT_RATE > 0:
    ratio_gap = abs(ptv_base_total / VAT_RATE - ptv_threshold_total)
    assert ratio_gap <= max(1e-6, 1e-9 * ptv_threshold_total), "PTV identity failed: base/VAT != thresholds (tolerance)"
else:
    print("VAT_RATE == 0 → skip PTV identity division check")

# ===== B) Flat per-capita: no-phase-out & implied tax-free consumption =====
# No-phase-out (rebate dollars): amount * hh_size
flat_base_each = panel["household_size"].astype(float) * float(FLAT_AMOUNT)
flat_base_total = wsum(flat_base_each, panel["household_weight"])

# "Implicit tax-free consumption": divide rebate by VAT rate (interpret amount ≈ VAT * allowance_per_capita)
if VAT_RATE > 0:
    flat_implicit_consumption_total = flat_base_total / VAT_RATE
else:
    flat_implicit_consumption_total = np.nan
    print("VAT_RATE == 0 → implicit consumption undefined; set to NaN")

# ===== Print results in billions =====
def b(x): 
    try:
        return round(float(x) / 1e9, 2)
    except:
        return x

print("\n--- VALIDATION (Billions of $) ---")
print(f"Poverty×VAT no-phase-out rebate total            : {b(ptv_base_total)}")
print(f"Poverty×VAT thresholds (no VAT multiplication)   : {b(ptv_threshold_total)}")
if VAT_RATE > 0:
    print(f"Check: (no-phase-out total) / VAT_RATE           : {b(ptv_base_total / VAT_RATE)}  (should match thresholds)")

print(f"\nFlat per-capita no-phase-out rebate total        : {b(flat_base_total)}")
if VAT_RATE > 0:
    print(f"Flat per-capita implicit tax-free consumption    : {b(flat_implicit_consumption_total)} (rebate / VAT_RATE)")
else:
    print("Flat per-capita implicit tax-free consumption    : n/a (VAT_RATE == 0)")

# ===== Extra sanity assertions =====
assert ptv_base_total >= 0 and flat_base_total >= 0, "Base totals must be non-negative"
print("\n✅ Validation complete.")


Params → {'vat.rate': 0.16, 'flat.amount': 1700.0, 'flat.single_start': 50000.0, 'flat.mfj_start': 100000.0, 'flat.rate': 0.05}

--- VALIDATION (Billions of $) ---
Poverty×VAT no-phase-out rebate total            : 53.97
Poverty×VAT thresholds (no VAT multiplication)   : 337.34
Check: (no-phase-out total) / VAT_RATE           : 337.34  (should match thresholds)

Flat per-capita no-phase-out rebate total        : 59.23
Flat per-capita implicit tax-free consumption    : 370.22 (rebate / VAT_RATE)

✅ Validation complete.
